In [1]:
import numpy as np

edges = np.loadtxt('filename.txt', dtype=int)
num_nodes = np.max(edges) + 1

adj_matrix = np.zeros((num_nodes, num_nodes))
for edge in edges:
    adj_matrix[edge[0], edge[1]] = 1
    adj_matrix[edge[1], edge[0]] = 1

print(adj_matrix.shape)


(1005, 1005)


In [2]:
import torch
x = torch.tensor(adj_matrix, dtype=torch.float)


c:\Users\hp\AppData\Local\Programs\Python\Python38\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
row, col = np.where(adj_matrix == 1)

edge_index = np.stack((row, col), axis=0)
print(edge_index.shape)

(2, 32770)


In [4]:
edge_index = torch.tensor(edge_index, dtype=torch.long)

In [5]:
c = np.loadtxt('f2.txt', dtype=int)

num_nodes = np.max(c) + 1

communities = np.zeros((num_nodes))
for e in range(len(c)):
    communities[e] = c[e][1]

print(communities)

[ 1.  1. 21. ...  1.  6. 22.]


In [6]:
y = torch.tensor(communities, dtype=torch.long)
print(y)

tensor([ 1,  1, 21,  ...,  1,  6, 22])


In [7]:
import networkx as nx

def draw(lables, nodes):
  np.random.seed = 30
  cmap = plt.cm.get_cmap('hsv', 42)
  node_labels = np.array(lables)
  norm = plt.Normalize(vmin=node_labels.min(), vmax=node_labels.max())
  color_map = cmap(norm(node_labels))

  G = nx.Graph()
  G.add_nodes_from(nodes)
  G.add_edges_from(edges)


  nx.draw_random(G, with_labels=False, node_size=3, width=0.01, node_color=color_map)
  return G

In [8]:
nodes_all = []
lbls_all = []
for i, lbl in enumerate(y):
  nodes_all.append(i)
  lbls_all.append(lbl)
  

In [9]:
# g = draw(lbls_all, nodes_all)

NameError: name 'plt' is not defined

In [10]:
print('♥ ♥ ♥ ♥ ♥ ♥ ♥ ♥ ♥ ♥ ♥ ♥ ♥')
print(f'Number of nodes: {g.number_of_nodes()}') 
print('------------------------')
print(f'Number of edges: {edge_index.shape[1]}') 
print('------------------------')
print(f'Maximum node degree: {max(dict(g.degree()).values())}') 
print('------------------------')
print(f'Minimum node degree: {min(dict(g.degree()).values())}') 
print('♥ ♥ ♥ ♥ ♥ ♥ ♥ ♥ ♥ ♥ ♥ ♥ ♥')

♥ ♥ ♥ ♥ ♥ ♥ ♥ ♥ ♥ ♥ ♥ ♥ ♥


NameError: name 'g' is not defined

In [11]:
pip install torch_geometric

     -------------------------------------- 616.2/616.2 kB 3.2 MB/s eta 0:00:00
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Created wheel for torch_geometric: filename=torch_geometric-2.3.0-py3-none-any.whl size=909920 sha256=5c8c426156e7f3babfaff6ad2a9fed833095e85ae354ee5de22a05f8d1e17c43
  Stored in directory: c:\users\hp\appdata\local\pip\cache\wheels\23\77\fd\db443fdbf7f6caad6924698316a88c8b8c063906125d846f8b
Successfully built torch_geometric
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.0.1 -> 23.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [12]:
import torch
from torch_geometric.data import Data
from torch_geometric.nn import GCNConv, global_max_pool, BatchNorm
from sklearn.model_selection import train_test_split
import torch.nn.functional as F

num_classes = len(torch.unique(y))

class Net(torch.nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = GCNConv(x.shape[1], 64)
        self.bn1 = BatchNorm(64)
        self.conv2 = GCNConv(64, 128)
        self.bn2 = BatchNorm(128)
        self.conv3 = GCNConv(128, num_classes)
        # self.fc = torch.nn.Linear(16, num_classes)
        # self.dropout = torch.nn.Dropout(0.5)

    def forward(self, x, edge):
        x = self.conv1(x, edge)
        x = self.bn1(x)
        x = F.relu(x)
        x = F.dropout(input=x, training=self.training)
        x = self.conv2(x, edge)
        x = self.bn2(x)
        x = F.relu(x)
        x = F.dropout(input=x, training=self.training)
        x = self.conv3(x, edge)

        return F.log_softmax(x, dim=1)


In [13]:
import torch.nn as nn
data = Data(x=x, edge_index=edge_index, y=y)
model = Net()
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
train_mask, test_mask = train_test_split(torch.arange(1004), test_size=0.8)
criterion = nn.CrossEntropyLoss()
print(train_mask.shape)

losses = []
model.train()
for epoch in range(200):
    optimizer.zero_grad()
    out = model(data.x, data.edge_index)
    # print(out[train_mask], y[train_mask])
    loss = criterion(input=out[train_mask], target=y[train_mask])
    loss.backward()
    optimizer.step()
    losses.append(loss.item())



torch.Size([200])


In [14]:
from bokeh.plotting import figure, show
from bokeh.models import ColumnDataSource
from bokeh.io import output_notebook
output_notebook()

source = ColumnDataSource(data=dict(x=list(range(len(losses))), y=losses))
plot = figure(title='Training Loss', x_axis_label='Epoch', y_axis_label='Loss')
line = plot.line('x', 'y', source=source)
line.glyph.line_alpha = 0.8
line.glyph.line_color = "blue"
output_notebook()
show(plot)

Loading BokehJS ...

Loading BokehJS ...

In [15]:
model.eval()
out = model(data.x, data.edge_index)
_, pred = out.max(dim=1)
correct = pred[test_mask].eq(data.y[test_mask]).sum().item()
accuracy = correct / len(data.y[test_mask])
print('Accuracy: {:.4f}'.format(accuracy))


Accuracy: 0.5908


In [16]:
classes = torch.unique(y)
num_classes = len(classes)
print(f"Number of classes: {num_classes}")

Number of classes: 42


In [17]:
import matplotlib.pyplot as plt
import numpy as np
def get_n_m(x):
    n = int(x ** 0.5)
    while x % n != 0:
        n -= 1
    return tuple(sorted((n, x // n)))

def visualize(edges, nodes, labels, communities, x):
    np.random.seed = 30
    cmap = plt.cm.get_cmap('hsv', 42)
    node_labels = np.array(labels)
    norm = plt.Normalize(vmin=node_labels.min(), vmax=node_labels.max())
    color_map = cmap(norm(node_labels))

    
    G = nx.Graph()
    G.add_nodes_from(nodes)
    G.add_edges_from(edges)

    pos = {}
    n = np.ceil(x ** 0.5)
    print(n)

    for node, community in enumerate(communities):
        x = (community % n) * 10  + np.random.random() * 7
        y = (community // n) * 10 + np.random.random() * 7
        pos[node] = [x, y]

    nx.draw(G, pos=pos, with_labels=False, node_size=3, width=0.01, node_color=color_map)

ImportError: cannot import name 'axes' from 'matplotlib' (c:\Users\hp\AppData\Local\Programs\Python\Python38\lib\site-packages\matplotlib\__init__.py)

In [18]:
nodes=[]
labels=[]
for i, lbl in enumerate(pred[test_mask]):
  nodes.append(i)
  labels.append(lbl)

In [19]:
len(pred)

1005

In [21]:
data.y[test_mask]
nodes_y=[]
labels_y=[]
for i, lbl in enumerate(data.y[test_mask]):
  nodes_y.append(i)
  labels_y.append(lbl)
visualize([ ], nodes_y, labels_y, labels_y, 42)

NameError: name 'visualize' is not defined

In [22]:
visualize([ ], nodes, labels_y, labels, max(pred))


NameError: name 'visualize' is not defined

In [23]:
from sklearn.metrics.cluster import normalized_mutual_info_score
nmi_score_gnn = normalized_mutual_info_score(labels_y, labels)

In [24]:
print(f'Normalized mutual info score: {nmi_score_gnn}')

Normalized mutual info score: 0.6685502362448683


In [25]:
pip install python-louvain


     -------------------------------------- 204.6/204.6 kB 2.5 MB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for python-louvain: filename=python_louvain-0.16-py3-none-any.whl size=9394 sha256=cad0d5278db825365eea7d29db1fb2452a2a91c18af27efbc99135384cad6ff6
  Stored in directory: c:\users\hp\appdata\local\pip\cache\wheels\af\40\91\ef3635f7d3ddbd889b5af9f1ac22b8d30c48d6f744cd74a6d1
Successfully built python-louvain



[notice] A new release of pip is available: 23.0.1 -> 23.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [29]:
import community as community_lv
G = nx.Graph()
G.add_nodes_from(nodes_y)
G.add_edges_from(edges[test_mask])
modularity_score = community_lv.modularity({i: pred[test_mask][i] for i in range(len(pred[test_mask]))}, G)
print("Modularity score: ", modularity_score)

Modularity score:  0.0807615603028665


In [33]:
edges[test_mask]

array([[160, 187],
       [448, 140],
       [ 41, 202],
       ...,
       [282, 424],
       [ 13, 301],
       [440, 413]])